In [1]:
### 라이브러리 import 
import matplotlib.pyplot as plt
import librosa.display
import librosa
import numpy as np
import glob
import os
import multiprocessing
import pandas as pd
import tensorflow as tf

from os.path import join
from tensorflow import keras

from keras.models import Sequential
from keras.layers import LSTM, Conv1D, Conv2D, MaxPooling1D, MaxPooling2D, Bidirectional, Flatten, GRU, Dense, Dropout, GlobalAveragePooling1D, BatchNormalization
from sklearn.model_selection import train_test_split
from sklearn.preprocessing import MinMaxScaler
from tensorflow.keras.preprocessing.sequence import pad_sequences

In [2]:
df_train = pd.read_csv('D:\pknu_6\daic-woz!/train_split.csv')
df_train = df_train[['Participant_ID','PHQ8_Binary']]
df_train

,Participant_ID,PHQ8_Binary
0,303,0
1,304,0
2,305,0
3,310,0
4,312,0
...,...,...
102,485,0
103,486,0
104,487,0
105,488,0


In [3]:
df_dev = pd.read_csv('D:\pknu_6\daic-woz!/dev_split .csv')
df_dev = df_dev[['Participant_ID','PHQ8_Binary']]
df_dev

,Participant_ID,PHQ8_Binary
0,302,0
1,307,0
2,331,0
3,335,1
4,346,1
5,367,1
6,377,1
7,381,1
8,382,0
9,388,1


In [4]:
df_train = pd.concat([df_train,df_dev], ignore_index=True)

In [5]:
df_train['PHQ8_Binary'].value_counts()

0    100
1     42
Name: PHQ8_Binary, dtype: int64

In [6]:
# 디렉토리 지정
base_folder_path='D:\pknu_6\daic-woz!\interim'

# 모든 파일의 경로와 라벨을 저장할 리스트를 초기화합니다.
data = []

# 디렉토리와 하위 디렉토리를 순회하며 모든 파일의 경로를 찾습니다.
for dirpath, dirnames, filenames in os.walk(base_folder_path):
     for filename in filenames:
            if filename.endswith(".wav"):
                # 파일의 전체 경로를 생성합니다.
                file_path = os.path.join(dirpath, filename)
                data.append(file_path)


# 리스트를 데이터 프레임으로 변환합니다.
df = pd.DataFrame(data, columns=['file_path'])
df

,file_path
0,D:\pknu_6\daic-woz!\interim\P300\P300_no_silen...
1,D:\pknu_6\daic-woz!\interim\P301\P301_no_silen...
2,D:\pknu_6\daic-woz!\interim\P302\P302_no_silen...
3,D:\pknu_6\daic-woz!\interim\P303\P303_no_silen...
4,D:\pknu_6\daic-woz!\interim\P304\P304_no_silen...
...,...
180,D:\pknu_6\daic-woz!\interim\P487\P487_no_silen...
181,D:\pknu_6\daic-woz!\interim\P488\P488_no_silen...
182,D:\pknu_6\daic-woz!\interim\P490\P490_no_silen...
183,D:\pknu_6\daic-woz!\interim\P491\P491_no_silen...


In [7]:
df_train['file_path'] = 0
for i in range(len(df_train['Participant_ID'])):
    k = df_train['Participant_ID'][i]
    for j in df['file_path']:
        if j == f'D:\pknu_6\daic-woz!\interim\\P{k}\\P{k}_no_silence.wav':
            print(j)
            df_train['file_path'][i] = j
df_train

D:\pknu_6\daic-woz!\interim\P303\P303_no_silence.wav
D:\pknu_6\daic-woz!\interim\P304\P304_no_silence.wav
D:\pknu_6\daic-woz!\interim\P305\P305_no_silence.wav
D:\pknu_6\daic-woz!\interim\P310\P310_no_silence.wav
D:\pknu_6\daic-woz!\interim\P312\P312_no_silence.wav
D:\pknu_6\daic-woz!\interim\P313\P313_no_silence.wav
D:\pknu_6\daic-woz!\interim\P315\P315_no_silence.wav
D:\pknu_6\daic-woz!\interim\P316\P316_no_silence.wav
D:\pknu_6\daic-woz!\interim\P317\P317_no_silence.wav
D:\pknu_6\daic-woz!\interim\P318\P318_no_silence.wav
D:\pknu_6\daic-woz!\interim\P319\P319_no_silence.wav
D:\pknu_6\daic-woz!\interim\P320\P320_no_silence.wav
D:\pknu_6\daic-woz!\interim\P321\P321_no_silence.wav
D:\pknu_6\daic-woz!\interim\P322\P322_no_silence.wav
D:\pknu_6\daic-woz!\interim\P324\P324_no_silence.wav
D:\pknu_6\daic-woz!\interim\P325\P325_no_silence.wav
D:\pknu_6\daic-woz!\interim\P326\P326_no_silence.wav
D:\pknu_6\daic-woz!\interim\P327\P327_no_silence.wav
D:\pknu_6\daic-woz!\interim\P328\P328_no_silen

,Participant_ID,PHQ8_Binary,file_path
0,303,0,D:\pknu_6\daic-woz!\interim\P303\P303_no_silen...
1,304,0,D:\pknu_6\daic-woz!\interim\P304\P304_no_silen...
2,305,0,D:\pknu_6\daic-woz!\interim\P305\P305_no_silen...
3,310,0,D:\pknu_6\daic-woz!\interim\P310\P310_no_silen...
4,312,0,D:\pknu_6\daic-woz!\interim\P312\P312_no_silen...
...,...,...,...
137,483,1,D:\pknu_6\daic-woz!\interim\P483\P483_no_silen...
138,484,0,D:\pknu_6\daic-woz!\interim\P484\P484_no_silen...
139,489,0,0
140,490,0,D:\pknu_6\daic-woz!\interim\P490\P490_no_silen...


In [8]:
df_train = df_train[df_train['file_path'] != 0]

In [9]:
df_train['PHQ8_Binary'].value_counts()

0    99
1    40
Name: PHQ8_Binary, dtype: int64

In [10]:
class_counts = {

    0: 99,
    1: 40,  }

# 클래스 가중치 조정
total_samples = sum(class_counts.values())
class_weights = {cls: total_samples / count for cls, count in class_counts.items()}

# Normalize weights
min_weight = min(class_weights.values())
class_weights = {cls: weight / min_weight for cls, weight in class_weights.items()}


In [11]:
class_weights

{0: 1.0, 1: 2.475}

In [12]:
df_train

,Participant_ID,PHQ8_Binary,file_path
0,303,0,D:\pknu_6\daic-woz!\interim\P303\P303_no_silen...
1,304,0,D:\pknu_6\daic-woz!\interim\P304\P304_no_silen...
2,305,0,D:\pknu_6\daic-woz!\interim\P305\P305_no_silen...
3,310,0,D:\pknu_6\daic-woz!\interim\P310\P310_no_silen...
4,312,0,D:\pknu_6\daic-woz!\interim\P312\P312_no_silen...
...,...,...,...
136,482,0,D:\pknu_6\daic-woz!\interim\P482\P482_no_silen...
137,483,1,D:\pknu_6\daic-woz!\interim\P483\P483_no_silen...
138,484,0,D:\pknu_6\daic-woz!\interim\P484\P484_no_silen...
140,490,0,D:\pknu_6\daic-woz!\interim\P490\P490_no_silen...


In [13]:
def split_audio_and_targets(filename, target, chunk_length=5):
    # Load the audio file
    y, sr = librosa.load(filename, sr=44100)

    # Calculate the number of 1-minute chunks
    total_length = librosa.get_duration(y=y, sr=sr)
    num_chunks = int(total_length / chunk_length)

    # Split the audio
    audio_chunks = []
    target_chunks = []
    for i in range(num_chunks):
        start = i * chunk_length * sr
        end = (i+1) * chunk_length * sr
        audio_chunk = y[start:end]
        audio_chunks.append(audio_chunk)
        target_chunks.append(target)  # Repeat the target value

    # If there are any leftovers, pad and add them as well
    if total_length > chunk_length * num_chunks:
        start = num_chunks * chunk_length * sr
        audio_chunk = np.pad(y[start:], (0, start + chunk_length * sr - len(y)))
        audio_chunks.append(audio_chunk)
        target_chunks.append(target)  # Repeat the target value for the padded chunk

    return audio_chunks, target_chunks


In [14]:
audio_load = []
train_target_list = []
for file_path, target in zip(df_train['file_path'],df_train['PHQ8_Binary']):
    audio_chunks, target_chunks = split_audio_and_targets(file_path, target)
    for j in range(len(audio_chunks)):
        audio_load.append(audio_chunks[j])
        train_target_list.append(target_chunks[j])

In [15]:
print(audio_load,train_target_list)

[array([ 0.00169721,  0.00209262,  0.00186246, ..., -0.00482857,
       -0.00347649, -0.00233926], dtype=float32), array([-0.00132211, -0.00037649,  0.00045462, ..., -0.00170078,
       -0.00201722, -0.00163871], dtype=float32), array([-8.1183889e-04, -4.9851427e-05,  2.5078777e-04, ...,
       -6.5327021e-03, -6.4725694e-03, -6.5290937e-03], dtype=float32), array([-0.00664584, -0.00674841, -0.00679879, ...,  0.00340601,
        0.00380878,  0.00414474], dtype=float32), array([0.00439901, 0.00465666, 0.00501383, ..., 0.00064944, 0.00071028,
       0.00083327], dtype=float32), array([ 0.00095946,  0.00101592,  0.00098288, ..., -0.00265321,
       -0.00251136, -0.00240084], dtype=float32), array([-0.00235563, -0.00237369, -0.00241632, ...,  0.0036738 ,
        0.00373846,  0.00372229], dtype=float32), array([ 0.00362958,  0.00353142,  0.00351303, ..., -0.00697522,
       -0.00640805, -0.00565034], dtype=float32), array([-0.00479991, -0.00399507, -0.00332787, ..., -0.00137846,
       -0.0

In [16]:
import librosa
import numpy as np
import parselmouth

def extract_features(audio_file):
    sr = 44100
    # Preemphasis
    y_pre = librosa.effects.preemphasis(audio_file, coef=0.97)

    # MFCC and Mel Spectrogram parameters
    n_fft = 1024
    hop_length = 256
    win_length = 512
    window = 'hamming'
    n_mels = 128
    n_mfcc = 64

    # Extract MFCC features
    mfccs = librosa.feature.mfcc(y=y_pre, win_length = win_length , sr=sr, n_mfcc=n_mfcc, n_mels=n_mels, n_fft=n_fft, hop_length=hop_length, window=window)

    # Extract Mel Spectrogram features
    mel_spectrogram = librosa.feature.melspectrogram(y=y_pre, sr=sr, n_fft=n_fft, hop_length=hop_length, n_mels=n_mels, window=window, win_length=win_length)
    mel_spectrogram_db = librosa.power_to_db(mel_spectrogram)

    # Load audio file with Parselmouth
    sound = parselmouth.Sound(audio_file)

    # Extract pitch
    pitch = sound.to_pitch(time_step=hop_length/sr)
    pitch_values = pitch.selected_array['frequency']

    # Extract intensity
    intensity = sound.to_intensity(time_step=hop_length/sr)
    intensity_values = intensity.values[0]

    # Interpolate pitch and intensity to match the length of MFCCs
    pitch_interp = np.interp(np.linspace(0, len(pitch_values), mfccs.shape[1]), np.arange(len(pitch_values)), pitch_values)
    intensity_interp = np.interp(np.linspace(0, len(intensity_values), mfccs.shape[1]), np.arange(len(intensity_values)), intensity_values)

    # Normalize MFCCs, pitch, intensity and Mel Spectrogram
    mfccs = (mfccs - np.min(mfccs)) / (np.max(mfccs) - np.min(mfccs))
    mel_spectrogram_db = (mel_spectrogram_db - np.min(mel_spectrogram_db)) / (np.max(mel_spectrogram_db) - np.min(mel_spectrogram_db))

    if not np.isnan(pitch_interp).all() and np.nanmin(pitch_interp) != np.nanmax(pitch_interp):
        pitch_interp = (pitch_interp - np.nanmin(pitch_interp)) / (np.nanmax(pitch_interp) - np.nanmin(pitch_interp))
    else:
        pitch_interp = np.zeros_like(pitch_interp)

    intensity_interp = (intensity_interp - np.min(intensity_interp)) / (np.max(intensity_interp) - np.min(intensity_interp))

    # Stack MFCCs, Mel Spectrogram, pitch and intensity features
    features = np.vstack([mfccs, mel_spectrogram_db, pitch_interp, intensity_interp])

    return features


In [17]:
audio_features = []
for i in range(1):
    features = extract_features(audio_load[i])
    audio_features.append(features)

In [18]:
audio_features[0].shape

(194, 862)

In [19]:
def create_dataset(file_paths, labels):
    def generator():
        for file_path, label in zip(file_paths, labels):
            yield extract_features(file_path), label

    dataset = tf.data.Dataset.from_generator(generator, 
                                             output_signature=(tf.TensorSpec(shape=(194, 862), dtype=tf.float32),
                                                              tf.TensorSpec(shape=(), dtype=tf.int32)))
    return dataset


In [20]:
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Conv2D, MaxPooling2D, Flatten, Dense, BatchNormalization, Dropout, ReLU, Softmax

model = Sequential()

# 1. conv block
model.add(Conv2D(16, (3,3), padding='same', activation='relu', input_shape=(194, 862, 1)))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# 2. conv block
model.add(Conv2D(32, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# 3. conv block
model.add(Conv2D(64, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# 4. conv block
model.add(Conv2D(128, (3,3), padding='same', activation='relu'))
model.add(BatchNormalization())
model.add(MaxPooling2D(pool_size=(2, 2)))
model.add(Dropout(0.3))

# flatten the output of the conv block
model.add(Flatten())

# output layer
model.add(Dense(1, activation='sigmoid'))

model.compile(loss='binary_crossentropy', optimizer='adam', metrics=['accuracy'])


In [60]:

checkpoint_cb = keras.callbacks.ModelCheckpoint("C:/pknu_6/model_w3_weight/model_{epoch}.h5", save_best_only=False)
early_stopping_cb = keras.callbacks.EarlyStopping(patience=200,
                                                 restore_best_weights=True)

In [61]:
model.load_weights("C:\pknu_6\model_w2_weight/model_115.h5")

In [62]:
# 파일 경로와 레이블을 리스트로 불러옵니다.
file_paths = audio_load
labels = train_target_list

# 훈련 데이터와 테스트 데이터를 분할합니다.
train_file_paths, val_file_paths, train_labels, val_labels =\
train_test_split(file_paths, labels, test_size=0.2, random_state=42, stratify=labels)

# 데이터 생성기를 생성합니다.
train_dataset = create_dataset(train_file_paths, train_labels)
val_dataset = create_dataset(val_file_paths, val_labels)

# 모델을 훈련시킵니다.
history = model.fit(train_dataset.batch(1), epochs=2000,class_weight=class_weights,
                    validation_data=val_dataset.batch(1),
                    callbacks=[checkpoint_cb, early_stopping_cb])


Epoch 1/2000
11955/11955 [==============================] - 789s 66ms/step - loss: 0.6301 - accuracy: 0.9307 - val_loss: 3.8706 - val_accuracy: 0.7451
Epoch 2/2000
 3862/11955 [========>.....................] - ETA: 6:39 - loss: 0.6131 - accuracy: 0.9319

KeyboardInterrupt: 

In [22]:
import os
import keras

# 파일 경로와 레이블을 리스트로 불러옵니다.
file_paths = audio_load
labels = train_target_list

# 훈련 데이터와 테스트 데이터를 분할합니다.
train_file_paths, val_file_paths, train_labels, val_labels =\
train_test_split(file_paths, labels, test_size=0.2, random_state=42, stratify=labels)

# 데이터 생성기를 생성합니다.
train_dataset = create_dataset(train_file_paths, train_labels)
val_dataset = create_dataset(val_file_paths, val_labels)

# 가중치 파일들이 저장된 디렉토리
weights_dir = 'C:/pknu_6/model_w2_weight/'

results = []

for i in range(50,100):
    model.load_weights(f'C:/pknu_6/model_w2_weight/model_{i}.h5')
        
    # Train set evaluation
    train_loss, train_accuracy = model.evaluate(train_dataset.batch(1))

    # Validation set evaluation
    val_loss, val_accuracy = model.evaluate(val_dataset.batch(1))

    results.append({
        'filename': f'model_{i}.h5',
        'train_loss': train_loss,
        'train_accuracy': train_accuracy,
        'val_loss': val_loss,
        'val_accuracy': val_accuracy
    })


# # 각 가중치 파일을 불러와서 모델의 성능을 평가합니다.
# for filename in os.listdir(weights_dir):
#     if filename.endswith(".h5"):  # 가중치 파일의 확장자가 .h5
#         model.load_weights(os.path.join(weights_dir, filename))
        
#         # Train set evaluation
#         train_loss, train_accuracy = model.evaluate(train_dataset.batch(1))
        
#         # Validation set evaluation
#         val_loss, val_accuracy = model.evaluate(val_dataset.batch(1))
        
#         results.append({
#             'filename': filename,
#             'train_loss': train_loss,
#             'train_accuracy': train_accuracy,
#             'val_loss': val_loss,
#             'val_accuracy': val_accuracy
#         })

# Save the results to a CSV file
import pandas as pd

results_df = pd.DataFrame(results)
results_df.to_csv('model_evaluations1.csv', index=False)


2989/2989 [==============================] - 194s 65ms/step - loss: 1.2139 - accuracy: 0.8193
   2694/Unknown - 175s 65ms/step - loss: 0.9277 - accuracy: 0.8259

IOPub message rate exceeded.
The notebook server will temporarily stop sending output
to the client in order to avoid crashing it.
To change this limit, set the config variable
`--NotebookApp.iopub_msg_rate_limit`.

Current values:
NotebookApp.iopub_msg_rate_limit=1000.0 (msgs/sec)
NotebookApp.rate_limit_window=3.0 (secs)



In [20]:
import pandas as pd

results_df = pd.DataFrame(results)
results_df.to_csv('model_evaluations.csv', index=False)


In [52]:
import matplotlib.pyplot as plt

# 정확도
plt.figure(figsize=(12, 6))
plt.plot(history.history['accuracy'], label='Train Accuracy')
plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.savefig('C:\pknu_6\woiz_mel_mfcc_history/accuracy_graph1.png')  # 그래프를 이미지 파일로 저장
plt.close()  # 현재 그래프를 닫음

# 손실
plt.figure(figsize=(12, 6))
plt.plot(history.history['loss'], label='Train Loss')
plt.plot(history.history['val_loss'], label='Validation Loss')
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(loc='upper left')
plt.savefig('C:\pknu_6\woiz_mel_mfcc_history/loss_graph1.png')  # 그래프를 이미지 파일로 저장
plt.close()  # 현재 그래프를 닫음


NameError: name 'history' is not defined

<Figure size 1200x600 with 0 Axes>

In [53]:
import pandas as pd

# 히스토리 객체를 pandas DataFrame으로 변환
history_df = pd.DataFrame(history.history)

# DataFrame을 CSV 파일로 저장
history_df.to_csv('C:\pknu_6\woiz_mel_mfcc_history/model_history1.csv', index=False)


NameError: name 'history' is not defined

In [49]:
df12345 = pd.read_csv('C:\pknu_6\woiz_mel_mfcc_history/model_history1.csv')

FileNotFoundError: [Errno 2] No such file or directory: 'C:\\pknu_6\\woiz_mel_mfcc_history/model_history1.csv'